<a href="https://colab.research.google.com/github/hyojunyee/kita_2404/blob/main/m3_%EB%B6%84%EC%84%9D%20%EB%9D%BC%EC%9D%B4%EB%B8%8C%EB%9F%AC%EB%A6%AC/Trainging/pandas_%EC%8B%A4%EC%8A%B5_train_0604_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 지금 넣어준건 GroupSize와 FarePerPerson, Title에 따라서 bool 값을 넣어줬다.
- cabin을 통해 좌석이 있냐 없냐로 deck에 null값을 넣어서 좌석여부에 따라 생존률을 결측값에 넣었으나, 너무 많은 불확실한 데이터가 들어가서 오히려 부정확한 데이터가 나와버렸다.
- Accuracy: 0.8547486033519553

Task4. train.csv 파일로 생존자 예측 모델 생성을 위한 분석용 데이터 셋을 작성하세요

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# 데이터 불러오기
file_path = '/content/drive/MyDrive/kdt_240424/m3_분석라이브러리/m_3_datas/train.csv'
# file_path = 'train.csv'
df = pd.read_csv(file_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# print(df.info())

In [ ]:
# print(df.head())

In [ ]:
# 성별을 숫자로 바꿈
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})

In [ ]:
# # 결측치 확인
# print(df.isnull().sum())

In [ ]:
# df_sorted = df.sort_values(by='Fare', ascending=False)
# df_sorted

In [ ]:
def extract_title(name):
    return name.split(',')[1].split('.')[0].strip()

# 'Title' 열 추가
df['Title'] = df['Name'].apply(extract_title)

# 호칭별 나이 평균 계산
title_age_mean = df.groupby('Title')['Age'].mean()

# 결측치를 호칭별 평균값으로 대체
for title, age_mean in title_age_mean.items():
    df.loc[(df['Age'].isnull()) & (df['Title'] == title), 'Age'] = age_mean

In [ ]:
# df.info()

In [ ]:
# # Cabin에서 Deck 정보를 추출하고 누락된 값을 채웁니다. pclass랑 표값에 따라서 자리가 있고 없을 수 있음.
# def fill_deck(pclass, deck_mode):
#     if pd.isna(pclass):
#         return deck_mode
#     return pclass

# # 캐빈에서 데크 정보 추출
# df['Deck'] = df['Cabin'].apply(lambda x: x[0] if pd.notna(x) else np.nan)

# # 각 Pclass에 대해 가장 일반적인 데크를 계산
# deck_mode_per_pclass = df.groupby('Pclass')['Deck'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else 'U')

# # 누락된 값 채우기
# df['Deck'] = df.apply(lambda row: fill_deck(row['Deck'], deck_mode_per_pclass[row['Pclass']]), axis=1)

# # 결과 확인
# print(df['Deck'].value_counts())

In [ ]:
# 1인당 요금을 계산
df['GroupSize'] = df.groupby('Ticket')['Ticket'].transform('count')
df['FarePerPerson'] = df['Fare'] / df['GroupSize']

In [ ]:
# Embarked에서 누락된 값을 처리
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)
df['Embarked'] = df['Embarked'].map({'C': 0, 'Q': 1, 'S': 2})

In [ ]:
# # 결측치 처리 (예: Age 열의 결측치를 평균으로 대체)
# df['Age'].fillna(df['Age'].mean(), inplace=True)

In [ ]:
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)

In [ ]:
# Age_group을 6개 범주로 확대
bins = [0,5,12,18,35,60,100]
labels = ['Infant','child','Teenager','Young Adult','Adult','Senior']
df['Age_group'] = pd.cut(df['Age'], bins=bins, labels=labels)
df.drop(columns=['Age'], inplace=True)
# df.head()

In [ ]:
# Fare를 3개의 범주로 변환
bins = [0, 25, 50, 600]
labels = ['Low', 'Medium', 'High']
df['Fare_bin'] = pd.cut(df['Fare'], bins=bins, labels=labels, include_lowest=True)

# 각 범주의 개수 출력
# fare_bin_counts = df['Fare_bin'].value_counts()
# print(fare_bin_counts)

In [ ]:
# One-Hot Encoding
# 범주형 변수를 더미 변수로 변환할 열 목록
categorical_columns = ['Age_group', 'Fare_bin', 'Embarked']

# 반복문을 사용하여 범주형 변수를 더미 변수로 변환
for column in categorical_columns:
    # 첫번쨰 카테고리는 drop_first=True로 제거
    df = pd.get_dummies(df, columns=[column])

In [ ]:
# df.info()

In [ ]:
# SibSp와 Parch 합쳐서 가족 규모 변수 생성
df['family_size'] = df['SibSp'] + df['Parch'] + 1   # 자기 자신 포함
df.drop(columns=['SibSp', 'Parch'], inplace=True)

In [ ]:
# df.info()

In [ ]:
# Name에 Title 값을 원핫 인코딩
df = pd.get_dummies(df, columns=['Title'])

In [ ]:
# # Deck 값을 원핫 인코딩
# df = pd.get_dummies(df, columns=['Deck'])

In [ ]:
# df.info()

In [ ]:
# 필요 없는 열 제거
df.drop(columns = ['PassengerId','Name','Ticket','Cabin'], inplace=True)
df.head()

,Survived,Pclass,Sex,Fare,GroupSize,FarePerPerson,Age_group_Infant,Age_group_child,Age_group_Teenager,Age_group_Young Adult,...,Title_Master,Title_Miss,Title_Mlle,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rev,Title_Sir,Title_the Countess
0,0,3,0,7.2500,1,7.2500,False,False,False,True,...,False,False,False,False,True,False,False,False,False,False
1,1,1,1,71.2833,1,71.2833,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
2,1,3,1,7.9250,1,7.9250,False,False,False,True,...,False,True,False,False,False,False,False,False,False,False
3,1,1,1,53.1000,2,26.5500,False,False,False,True,...,False,False,False,False,False,True,False,False,False,False
4,0,3,0,8.0500,1,8.0500,False,False,False,True,...,False,False,False,False,True,False,False,False,False,False


In [ ]:
# df.drop(columns = ['Cabin'], inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 36 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Survived               891 non-null    int64  
 1   Pclass                 891 non-null    int64  
 2   Sex                    891 non-null    int64  
 3   Fare                   891 non-null    float64
 4   GroupSize              891 non-null    int64  
 5   FarePerPerson          891 non-null    float64
 6   Age_group_Infant       891 non-null    bool   
 7   Age_group_child        891 non-null    bool   
 8   Age_group_Teenager     891 non-null    bool   
 9   Age_group_Young Adult  891 non-null    bool   
 10  Age_group_Adult        891 non-null    bool   
 11  Age_group_Senior       891 non-null    bool   
 12  Fare_bin_Low           891 non-null    bool   
 13  Fare_bin_Medium        891 non-null    bool   
 14  Fare_bin_High          891 non-null    bool   
 15  Embark

In [ ]:
# # 예시로 선택할 열 목록
# columns_of_interest = ['Survived','Age','Pclass','Sex','SibSp','Parch','Fare']

# # 선택한 열들로 새로운 데이터프레임 생성
# selected_df = df[columns_of_interest]

In [ ]:
# correlation_matrix = df.corr()

# # Heatmap으로 시각화
# plt.figure(figsize=(10, 8))
# sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
# plt.title("Correlation Matrix")
# plt.show()

In [ ]:
# 타겟 변수와 피처 변수 설정
X = df.drop('Survived', axis=1)
y = df['Survived']

# 학습 데이터와 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 크기 확인
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# 랜덤 포레스트 모델 학습
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# 성능 평가
print("Accuracy:", accuracy_score(y_test, y_pred))
# print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
# print("Classification Report:\n", classification_report(y_test, y_pred))

(712, 35) (179, 35) (712,) (179,)
Accuracy: 0.8547486033519553
